In [ ]:
# import set 
using Turing
using Distances
using LinearAlgebra
using LimberJack
using Interpolations
using GaussianProcess
using Plots
using QuadGK
using BlockDiagonals
using PrettyTables

In [2]:
# fs8 data
function fs8_DESI()

z  = [0.3, 0.51, 0.71, 0.92, 1.32, 1.49]

data  = [0.37784, 0.515897, 0.48384, 0.422208, 0.37468, 0.435]

fs8_upper = [0.4723, 0.572693, 0.539136, 0.470586, 0.418064, 0.48]
fs8_lower = [0.28338, 0.449635, 0.428544, 0.378228, 0.343128, 0.39]
err_up = fs8_upper .- data
err_low = data .- fs8_lower
err_tot = fs8_upper .- fs8_lower

cov = zeros(length(z), length(z))
    for i in 1:length(z)
        cov[i, i] = err_tot[i]^2
    end
    
    return (data_name = "DESI_fs8", z = z, data = data, cov = cov)
end

fs8_DESI (generic function with 1 method)

In [ ]:
# DESI data
z = [0.3, 0.51, 0.71, 0.92, 1.32, 1.49]
data_dvrd = [7.788174, 12.514437, 15.675560, 19.676985, 23.861806, 25.708520]
data_dhdm = [3.053800, 1.637266, 1.165867, 0.844996, 0.470709, 0.426508]
data_fss8 = [0.377174, 0.513635, 0.483623, 0.422164, 0.376715, 0.434858]
data_DESI = [data_dvrd; data_dhdm; data_fss8]

# Full 18x18 Covariance Matrix for DESI 2024 Year 1 
# Variables: [DV/rd, DH/DM, fσs8] per redshift bin
# Source: arXiv:2411.12021 Appendix A
C_total = zeros(18, 18);

# BGS z=0.30
C_total[1:3, 1:3] = 1e-4 * [
    1314.664401 -142.669742  109.427163;
    -142.669742  829.170940 -158.101737;
    109.427163 -158.101737   88.510877
]

# LRG1 z=0.51
C_total[4:6, 4:6] = 1e-4 * [
    541.309833   48.425593   -4.652853;
    48.425593   97.249820  -34.923265;
    -4.652853  -34.923265   41.295470
]

# LRG2 z=0.71
C_total[7:9, 7:9] = 1e-4 * [
    762.457717   39.781004   -1.896006;
    39.781004   36.344098  -17.341350;
    -1.896006  -17.341350   28.119682
]

# LRG3 z=0.92
C_total[10:12, 10:12] = 1e-4 * [
    847.499793   26.038900    3.257324;
    26.038900   16.251088  -10.044074;
    3.257324  -10.044074   22.370314
]

# ELG2 z=1.32
C_total[13:15, 13:15] = 1e-4 * [
2342.506886   26.159601   13.521001;
    26.159601   10.309303   -6.654663;
    13.521001   -6.654663   13.997473
]

# QSO z=1.49
C_total[16:18, 16:18] = 1e-4 * [
3013.788566   -2.205101   36.332110;
    -2.205101    5.845806   -6.747133;
    36.332110   -6.747133   19.785658
]

# correlation matrix
function cov_to_cor(covariance)
    std_devs = sqrt.(diag(covariance))
    return covariance ./ (std_devs * std_devs')
end

full_covariance = cov_to_cor(C_total);
#pretty_table(full_covariance)

cov_final = C_total
for k in 1:6
    cov_final[3*k,3*k] = fs8data.cov[k,k]
end

for h in 1:6
    cov_final[(3*h)-2,3*h] = cov_final[3*h,3*h]*full_covariance[(3*h)-2,3*h]
    cov_final[(3*h)-1,3*h] = cov_final[3*h,3*h]*full_covariance[(3*h)-1,3*h]
    cov_final[3*h,(3*h)-2] = cov_final[3*h,3*h]*full_covariance[(3*h)-2,3*h]
    cov_final[3*h,(3*h)-1] = cov_final[3*h,3*h]*full_covariance[(3*h)-1,3*h]
end

#pretty_table(cov_final)

3×3 Matrix{Float64}:
  0.301379    -0.00022051    0.00363321
 -0.00022051   0.000584581  -0.000674713
  0.00363321  -0.000674713   0.00197857

In [ ]:
function CC()
    z = [
        0.07, 0.09, 0.12, 0.17, 0.179, 0.199, 0.2,
        0.27, 0.28, 0.352, 0.38, 0.3802, 0.4,
        0.4004, 0.4247, 0.44, 0.4497, 0.47, 0.4783,
        0.48, 0.51, 0.593, 0.6, 0.61, 0.68, 0.73,
        0.781, 0.875, 0.88, 0.9, 1.037, 1.3,
        1.363, 1.43, 1.53, 1.75, 1.965
    ]

    data = [
        69.0, 69.0, 68.6, 83.0, 75.0, 75.0, 72.9,
        77.0, 88.8, 83.0, 81.5, 83.0, 95.0, 77.0,
        87.1, 82.6, 92.8, 89.0, 80.9, 97.0, 90.4,
        104.0, 87.9, 97.3, 92.0, 97.3, 105.0,
        125.0, 90.0, 117.0, 154.0, 168.0, 160.0,
        177.0, 140.0, 202.0, 186.5
    ]

    err = [
        19.6, 12.0, 26.2, 8.0, 4.0, 5.0, 29.6,
        14.0, 36.6, 14.0, 1.9, 13.5, 17.0, 10.2,
        11.2, 7.8, 12.9, 23.0, 9.0, 62.0, 1.9,
        13.0, 6.1, 2.1, 8.0, 7.0, 12.0, 17.0,
        40.0, 23.0, 20.0, 17.0, 33.6, 18.0,
        14.0, 40.0, 50.4
    ]

    cov = zeros(length(z), length(z))
    for i in 1:length(z)
        cov[i, i] = err[i]^2
    end

    return (
        data_name = "CC",
        z = z,
        data = data,
        cov = cov
    )
end
